In [31]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import torch

torch.set_printoptions(edgeitems=2)
# Sets the seed for generating random numbers. Returns a torch.Generator object.
torch.manual_seed(123)

In [32]:
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

In [33]:
from torchvision import datasets, transforms
data_path = '../data-unversioned/p1ch7/'
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(
            (0.4915, 0.4823, 0.4468),
            (0.2470, 0.2435, 0.2616)
        )
    ])
)

Files already downloaded and verified


In [34]:
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=False,
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(
            (0.4915, 0.4823, 0.4468),
            (0.2470, 0.2435, 0.2616)
        )
    ])
)

In [35]:
# 从cifar10里拿出飞机和鸟组成新的数据集，这个是datasets的子类
label_map = {0:0, 2:1}
class_names = ['airplane', 'bird']
cifar2 = [(img, label_map[label])
            for img, label in cifar10
            if label in [0, 2]]
cifar2_val = [(img, label_map[label])
                for img, label in cifar10_val
                if label in [0, 2]]

In [36]:
import torch.nn as nn 

n_out = 2 

model = nn.Sequential(
    nn.Linear(
        3072, # input features
        512, # hidden layer size
    ),
    nn.Tanh(),
    nn.Linear(
        512, # hidden layer size 
        n_out,
    )
)

In [37]:
def softmax(x):
    # Returns a new tensor with the exponential of the elements of the input tensor input.
    return torch.exp(x) / torch.exp(x).sum()

In [38]:
x = torch.tensor([1.0, 2.0, 3.0])

softmax(x)

tensor([0.0900, 0.2447, 0.6652])

In [39]:
softmax(x).sum()

tensor(1.)

In [40]:
softmax = nn.Softmax(dim=1) # 指定计算列数

x = torch.tensor([[1.0, 2.0, 3.0],
                [1.0, 2.0, 3.0]])

softmax(x), x.shape

(tensor([[0.0900, 0.2447, 0.6652],
         [0.0900, 0.2447, 0.6652]]),
 torch.Size([2, 3]))

In [41]:
model = nn.Sequential(
    nn.Linear(3072, 512),
    nn.Tanh(),
    nn.Linear(512, 2),
    nn.Softmax(dim=1)
)